In [ ]:
# import necessary libraries
import logging
import numpy as np
import os

from utils import FileManager
from Frame import Frame
from utils import ImageVisualizer
from utils import generic_o3d_vis

imgvis = ImageVisualizer()
logging.basicConfig(level=logging.ERROR)

In [ ]:
# choose the directory and the frame to be visualized
BASE_DIR = os.getcwd()
RUN_DIR_PATH = os.path.join(BASE_DIR, "sample_data/sample_run")

fm = FileManager(RUN_DIR_PATH)
pair = fm.get_pair(0)
frame = Frame(
    depth_path=pair.depth_path,
    color_path=pair.color_path,
    raw_timestamp=pair.raw_timestamp,
    recording_time_str=pair.recording_time_str
)

In [ ]:
# visualize the raw depth and color images
imgvis.depth_color_side_by_side(frame.depth, frame.color)

In [ ]:
# visualize the raw pcd
raw_pcd = frame.raw_pixels.create_pcd_from_pixels()
generic_o3d_vis(raw_pcd)

In [ ]:
# demonstration of the systemic depth error
bin_size = 25
min_depth = np.min(frame.depth)
max_depth = np.max(frame.depth)
bin_edges = np.arange(min_depth, max_depth + bin_size, bin_size)
print(f"bin_size: {bin_size}, min_depth: {min_depth}, max_depth: {max_depth}")

for i in reversed(range(3, len(bin_edges) - 1)):
    print(f"Processing bin {i + 1}/{len(bin_edges) - 1} with edges {bin_edges[i]} to {bin_edges[i + 1]}")
    bin_min = bin_edges[i]
    bin_max = bin_edges[i + 1]

    bin_pixels = frame.raw_pixels.get_pixels_in_range(bin_min, bin_max)
    bin_depth = bin_pixels.create_depth_from_pixels()
    bin_color = bin_pixels.create_color_from_pixels()
    imgvis.depth_color_side_by_side(bin_depth, bin_color)
    
    bin_fg_pixels = bin_pixels.get_foreground_pixels()
    bin_fg_depth = bin_fg_pixels.create_depth_from_pixels()
    bin_fg_color = bin_fg_pixels.create_color_from_pixels()
    imgvis.depth_color_side_by_side(bin_fg_depth, bin_fg_color)

    bin_bg_pixels = bin_pixels.get_background_pixels()
    bin_bg_depth = bin_bg_pixels.create_depth_from_pixels()
    bin_bg_color = bin_bg_pixels.create_color_from_pixels()
    imgvis.depth_color_side_by_side(bin_bg_depth, bin_bg_color)



In [ ]:
# background pixels
bg_pixels = frame.raw_pixels.get_background_pixels()

bg_depth_image = bg_pixels.create_depth_from_pixels()
bg_color_image = bg_pixels.create_color_from_pixels()
imgvis.depth_color_side_by_side(bg_depth_image, bg_color_image)

In [ ]:
# visualize all the background pixels as a point cloud with the fitted circle
geometries = []
for bg_circle in frame.bg_circles:
    bin_geometries = bg_circle.get_geometries()
    geometries.extend(bin_geometries)
generic_o3d_vis(geometries)

In [ ]:
# foreground pixels 
fg_pixels = frame.raw_pixels.get_foreground_pixels()

fg_depth_image = fg_pixels.create_depth_from_pixels()
fg_color_image = fg_pixels.create_color_from_pixels()

imgvis.depth_color_side_by_side(fg_depth_image, fg_color_image)


In [ ]:
# visualize all the foreground pixels as a point cloud with the fitted circle
geometries = []
for fg_circle in frame.fg_circles:
    bin_geometries = fg_circle.get_geometries()
    geometries.extend(bin_geometries)
generic_o3d_vis(geometries)